# variousCity指武汉、重庆、成都、南京、昆明
# variousClass 指根据指数和JRC，获取陆地和永久性水体的范围，未考虑季节性水体，在此类区域上选择样本

In [2]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [4]:
wuhan = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
chongqing = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/chongqing').first()).geometry()
chengdu = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/chengdu').first()).geometry()
nanjing = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/nanjing').first()).geometry()
kunming = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/kunming').first()).geometry()

roi = ee.FeatureCollection([wuhan,chongqing,chengdu,nanjing,kunming])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,6)

In [5]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 图像可视化参数
visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# 计算指数
def ndvi_mndwi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi)

# 计算影像裁剪后的面积
def area_after_clip(image):
    image_area = image.clip(wuhan).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

In [6]:
# 武汉
wuhan_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(wuhan) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

# 重庆
chongqing_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(chongqing) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

# 成都
chengdu_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(chengdu) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

# 南京
nanjing_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(nanjing) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

# 昆明
kunming_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(kunming) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

# 武汉+重庆+成都+南京+昆明
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

In [7]:
print(landsat_images.size().getInfo())

405


# 筛选得出采样影像

In [ ]:
# image = landsat_images.mosaic().clip(roi)
# image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
# areas = image_area.reduceRegion(**{
#     'reducer': ee.Reducer.sum(),
#     'geometry': roi.geometry(),
#     'scale': 30,
#     'maxPixels': 1e14
# })
# print(areas.getInfo())


# # 武汉
# wuhan_sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('area',170))
# dates = wuhan_sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
# print(sample_image.size().getInfo())
# print(dates.getInfo())

In [ ]:

wuhan_sample_image = wuhan_images.map(area_after_clip).filter(ee.Filter.gt('area',170))
print(wuhan_sample_image.size().getInfo())
# chongqing_sample_image = chongqing_images.map(area_after_clip).filter(ee.Filter.gt('area',7962))
# chengdu_sample_image = chengdu_images.map(area_after_clip).filter(ee.Filter.gt('area',1413))
# nanjing_sample_image = nanjing_images.map(area_after_clip).filter(ee.Filter.gt('area',658))
# kunming_sample_image = kunming_images.map(area_after_clip).filter(ee.Filter.gt('area',2012))
# sample_image = wuhan_sample_image.merge(chongqing_sample_image).merge(chengdu_sample_image).merge(nanjing_sample_image).merge(kunming_sample_image)
# # print(sample_image.size().getInfo())

# 各个数据集，不同水体类型范围的确定

##  基于指数的永久水体和季节水体

In [ ]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')

# 计算水体频率
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
vis = {'bands': ['count'], 'palette': ['ffffff', 'dd09ec'], 'min': 0.0, 'max': 35.0, 'opacity': 1.0}
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()

## 基于JRC的永久水体和季节水体

In [ ]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
# 水体分类；“0：陆地；1：季节性水体；2：永久性水体”
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# Map.addLayer(JRC_waterclass,{'palette':['white','red','blue'],'min':0,'max':2},"JRC waterclass")

# 各个水体数据集在永久性水体和季节性水体的公共部分（交集，intersection）

## 永久性/季节性 水体的交集

In [ ]:
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
# Map.addLayer(permanentWaterExtent,{'palette':['white','blue'],'min':0,'max':1},'permanent water extent')

landWaterExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)
# Map.addLayer(landWaterExtent,{'palette':['white','cyan'],'min':0,'max':1},'land extent')

# 样本采集函数的定义

In [ ]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi']
# 分类标签
label = 'waterclass'

def imageSample(image):
    permanent_image = image.clip(wuhan).updateMask(permanentWaterExtent)
    permanent = permanentWaterExtent.updateMask(permanent_image.select('B2').gt(0))
    land_image = image.clip(wuhan).updateMask(landWaterExtent)
    land = landWaterExtent.updateMask(land_image.select('B2').gt(0)).remap([1],[0]).rename('waterclass')
    points_permanentwater = permanent.sample(**{
        'region': wuhan,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    points_land = land.sample(**{
        'region': wuhan,
        'scale': 30,
        'numPixels': 50,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    all_points = points_permanentwater.merge(points_land)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30
    })
    return sample_points

In [ ]:
# wuhan_pointsCollection = wuhan_sample_image.map(imageSample).flatten()
# chongqing_pointsCollection = chongqing_sample_image.map(imageSample).flatten()
# chengdu_pointsCollection = chengdu_sample_image.map(imageSample).flatten()
# nanjing_pointsCollection = nanjing_sample_image.map(imageSample).flatten()
# kunming_pointsCollection = kunming_sample_image.map(imageSample).flatten()
# points_collection = wuhan_pointsCollection.merge(chongqing_pointsCollection).merge(chengdu_pointsCollection).merge(nanjing_pointsCollection).merge(kunming_pointsCollection)

# # print(points_collection.size().getInfo())

In [ ]:
wuhan_pointsCollection = wuhan_sample_image.map(imageSample).flatten()
print(wuhan_pointsCollection.size().getInfo())

# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

In [ ]:
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
#对Landsat-8进行分类
def training(image):
    return image.clip(roi).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
waterpixel_count = landsat_images.map(training).sum()
# Map.addLayer(waterpixel_count,{},"waterPixelCount")

In [ ]:
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

In [ ]:
validPixel = landsat_images.count().select('B3').clip(roi).rename('count')
waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"frequency")

In [ ]:
permanentwater = waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
Map.addLayer(permanentwater,{'palette':['blue']},"permanent water")

In [ ]:
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
permanent_water = waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
seasonal_water = ee.ImageCollection([waterfrequency.gte(0.25),waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
water_waterclass = ee.ImageCollection([permanent_water,seasonal_water,basemap]).sum()
# Map.addLayer(water_waterclass,{'palette':['white','red','blue'],'min':0,'max':2},"water waterclass")

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['0000ff']}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(permanentwater, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(recent)', fontsize=20)

In [ ]:
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
# use cartoee to get a map
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
# add a colorbar to the map using the visualization params we passed to the map
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# ticks=range(0,50,10)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(recent)', fontsize=20)

# 使用JRC数据集做交叉验证

## JRC

In [ ]:
# 使用JRC做验证
JRC_WaterClass = JRC_waterclass.rename('jrcwaterclass')
JRC_samplepoints = JRC_WaterClass.sample(**{
    'region': roi ,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))

waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',2))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

In [ ]:
# Overlay the points on the imagery to get training.
JRC_sample_point = water_waterclass.sampleRegions(**{
    'collection': JRC_samplepoints,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

## Maryland

In [ ]:
# 使用Maryland做验证
Maryland_WaterClass = Maryland_waterclass.rename('Maryland_waterclass')
Maryland_samplepoints = Maryland_WaterClass.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))

waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

waterclass = Maryland_samplepoints.filter(ee.Filter.eq('Maryland_waterclass',2))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

In [ ]:
# Overlay the points on the imagery to get training.
Maryland_sample_point = water_waterclass.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['Maryland_waterclass'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('Maryland_waterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

# 附录

In [ ]:
# Map.addLayer(sample_image.first().clip(roi),visParams,'image first')
permanent_image = sample_image.first().clip(roi).updateMask(permanentWaterExtent)
Map.addLayer(permanent_image,visParams,'permanent image')
permanent = permanentWaterExtent.updateMask(permanent_image.select('B2').gt(0))
Map.addLayer(permanent,{'palette':['white','blue'],'min':0,'max':1},'permanent')


season_image = sample_image.first().clip(roi).updateMask(seasonWaterExtent)
seasonal = seasonWaterExtent.updateMask(season_image.select('B2').gt(0))
Map.addLayer(seasonal,{'palette':['white','red'],'min':0,'max':1},'seasonal')

land_image = sample_image.first().clip(roi).updateMask(landWaterExtent)
land = landWaterExtent.updateMask(land_image.select('B2').gt(0)).remap([1],[0]).rename('waterclass')
Map.addLayer(land,{'palette':['white','cyan'],'min':0,'max':1},'land')

points_permanentwater = permanent.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print("permanent : {}".format(points_permanentwater.size().getInfo()))
points_seasonalwater = seasonal.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('seasonal : {}'.format(points_seasonalwater.size().getInfo()))

points_land = land.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('land : {}'.format(points_land.size().getInfo()))

In [ ]:
all_points = points_permanentwater.merge(points_seasonalwater).merge(points_land)
print(all_points.size().getInfo())
print(all_points.filter(ee.Filter.eq('waterclass',1)).size().getInfo())

In [ ]:
per_wuhan = landsat_images.mosaic().updateMask(permanentWaterExtent)
Map.addLayer(per_wuhan,visParams,"per wuhan")

In [ ]:
# histogram = index_water.select('waterclass').reduceRegion(**{
#     'reducer': ee.Reducer.histogram(100),
#     'geometry': roi.geometry(), 
#     'scale': 30,
#     'bestEffort': True
# })
# print(histogram.getInfo())

In [ ]:
dataset = ee.Image('JRC/GSW1_2/YearlyHistory/2018')

visualization = {
  'bands': ['waterClass'],
  'min': 0.0,
  'max': 3.0,
  'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
}
Map.addLayer(dataset.clip(roi), visualization, 'Water Class')